# S05 v2 — Harmonía funcional con music21 (roman numerals sobre coral)
**Duración:** 1 sesión (≈ 1 hora)  
**Enfoque (Armonía):** acordes reais + cifrado romano + lectura crítica

## 🎯 Obxectivos
- Usar `chordify()` para obter unha redución armónica
- Estimar tonalidade e aplicar cifrado romano
- Detectar cadencias simples (V–I, IV–V–I…)
- Comparar análise automática vs análise humana

> O importante non é “acertar sempre”, senón **interpretar**.


## 0) Setup


In [ ]:
!pip -q install music21
from music21 import corpus, roman, key
print("✅ listo")


## 1) Cargar obra e estimar tonalidade


In [ ]:
score = corpus.parse("bach/bwv66.6")
k = score.analyze("key")
print("Tonalidade estimada:", k)


## 2) Reducir a acordes (chordify)
`chordify()` crea unha capa de acordes aproximados ao que soa en cada momento.


In [ ]:
ch = score.chordify()
ch.show("text")


## 3) Cifrado romano sobre os acordes
Imos etiquetar os primeiros N acordes.

⚠️ Hai acordes de paso e notas non harmónicas:
por iso ás veces o cifrado “parece raro”.


In [ ]:
from music21 import chord as m21chord

def romanizar_stream(chordified, tonalidade, max_items=25):
    out = []
    kk = key.Key(str(tonalidade.tonic), tonalidade.mode)
    for c in chordified.recurse().getElementsByClass(m21chord.Chord)[:max_items]:
        try:
            rn = roman.romanNumeralFromChord(c, kk)
            out.append((c.offset, rn.figure, [p.name for p in c.pitches]))
        except Exception as e:
            out.append((c.offset, "?", [p.name for p in c.pitches]))
    return out

rows = romanizar_stream(ch, k, max_items=20)
rows[:10]


## 4) Detectar cadencias simples (heurística)
Buscamos patróns do tipo:
- V → I
- V7 → I
- ii → V → I (cando apareza)

Isto é aproximado, serve para *localizar* zonas de cadencia e logo revisar a man.


In [ ]:
def detectar_V_I(figs):
    hits = []
    for i in range(len(figs)-1):
        a = figs[i][1]
        b = figs[i+1][1]
        if a.startswith("V") and (b.startswith("I") or b.startswith("i")):
            hits.append((figs[i][0], a, "→", b))
    return hits

figs = rows
cad = detectar_V_I(figs)
cad[:10]


## 🧩 Tarefa (avaliable)
1) Escolle un rango pequeno (ex: 8–12 eventos) e:
- escribe os cifrados romanos
- marca se cres que hai notas de paso / retardos
2) Localiza polo menos **1 posible cadencia** e xustifícaa (2–3 liñas)

### Reflexión (6–8 liñas)
- En que momentos `music21` che axuda máis?
- En que momentos hai que desconfiar do automático?


In [ ]:
# ✅ Espazo de traballo: escolle outro max_items ou outra obra
# score2 = corpus.parse("bach/bwv269")
# ...
